<h1>A Hands-On Lab for the System Programmer</h1>
<ul>
    <li>You need to catch a run-away job that is generating SMF records and filling our logstream!</li>
    <li>SYSLOG isn’t helpful, we need to do some ad-hoc processing of SMF30 to figure this out.</li>
</ul>
<h5>Based off of <a href="http://mainframeinsights.com/dunnit-spark-can-help/">Who dunnit... Spark, can you help?!</a></h5>

We'll obtain our system's IP Address and our user password four our JDBC call

In [ ]:
/*************************************************************************************/
/* Read .system and .id_pass to obtain the system's IP address and password          */
/*************************************************************************************/
import scala.io.Source._
val zos_system = fromFile("/u/nmarion/data/.system", "IBM-1047").getLines.mkString
val pass = fromFile("/u/nmarion/data/.id_pass", "IBM-1047").getLines.mkString

Using Rocket's MDS and JDBC driver, we can read our SMF dataset and load it into a dataframe.

In [ ]:
/*************************************************************************************/
/* Create JDBC call with our credentials and specifying the virtual SMF table along  */
/* with our dataset we'd like to map over.                                           */ 
/*                                                                                   */
/* We have chosen the Identification Section of the SMF30 Record for our mapping     */
/*************************************************************************************/

val dfReader = spark.read.format("jdbc").option("driver", "com.rs.jdbc.dv.DvDriver")
dfReader.option("url", "jdbc:rs:dv://" + zos_system + ";DBTY=DVS;user=nmarion;password=" + pass)
dfReader.option("dbtable", "SMF_03000_SMF30ID__NMARION_AQHO_SMF30_D17055_T111328")
val df = dfReader.load()

Lets look at how many SMF records we have in this dataset.

In [ ]:
/*************************************************************************************/
/* Count the number of entries in dataframe. Expecting 1167                          */
/*************************************************************************************/
df.count()

Lets limit the amount of records to 250 and save our dataframe into a json file for later.

In [ ]:
/*************************************************************************************/
/* Later in the lab, we'll attempt to read a json file. Saving for later. If file    */
/* exists, overwrite it.                                                             */
/*************************************************************************************/
df.write.mode("overwrite").json("/u/nmarion/data/SMF30.json")

Take a look at our records we notice that there are a lot of columns.

In [ ]:
/*************************************************************************************/
/* The show() method displays the contents of the DataFrame. By default, only the    */
/* first 20 rows will be displayed. You may pass an integer to display more or less  */
/* rows.                                                                             */
/*                                                                                   */
/* In this case, we're displaying all the columns in the Identification Section of   */
/* SMF30 Records, which is too much to display on a single screen.                   */
/*************************************************************************************/
df.show()

Lets take a look at our schema which will describe our columns and we can find what we're interesting in.

In [ ]:
/*************************************************************************************/
/* The printSchema() method shows us the headers of the DataFrame in a clean format. */
/* It will show the header, the data type, and if it is nullable or not.             */
/*                                                                                   */
/* For this lab, we're interesting in the Job Name (SMF30JBN) and RACF User ID       */
/* (SMF30RUD).                                                                       */
/*************************************************************************************/
df.printSchema()

We can take a look at the SMF30JBN, SMF 30 Jobname, and the SMF30RUD, SMF 30 User ID and display them.

In [ ]:
/*************************************************************************************/
/* The select() method allows you to select specific columns in the DataFrame.       */
/*                                                                                   */
/* Here we've selected the Job Name (SMF30JBN) and RACF User ID (SMF30RUD) and will  */
/* only show the information we're interesting in.                                   */
/*************************************************************************************/
df.select("SMF30JBN", "SMF30RUD").show()

This doesn't quite give us an idea of who could be filling up our dataset as we only see the top 20. Lets group the same job names and the same users together and count them.

In [ ]:
/*************************************************************************************/
/* The groupBy() method allows us to specify a column to group by. Adding the count()*/
/* method, we create a new count column associated with each grouping. Below, we     */
/* create a jobs and users DataFrame with the counts for each.                       */
/*************************************************************************************/
val jobs = df.groupBy("SMF30JBN").count()
val users = df.groupBy("SMF30RUD").count()

Now that we've created our own dataframes with counts, lets display them in desc order and show the top 10. We see that the user SATURN and their jobs are making up the majority of our records.

In [ ]:
/*************************************************************************************/
/* The orderBy() method can sort by a given expression. Below we sort our jobs and   */
/* users DataFrames in descending order and limit to the top 10.                     */
/*************************************************************************************/
jobs.orderBy(jobs.col("count").desc).limit(10).show()
users.orderBy(users.col("count").desc).limit(10).show()

As mentioned previously you can also read json files. Previously we saved the smaller DataFrame. We can repeat our code and should get the same results.

In [ ]:
val jsonDf = spark.read.json("/u/nmarion/data/SMF30.json")
val jsonJobs = jsonDf.groupBy("SMF30JBN").count()
val jsonUsers = jsonDf.groupBy("SMF30RUD").count()
jsonJobs.orderBy(jsonJobs.col("count").desc).limit(10).show()
jsonUsers.orderBy(jsonUsers.col("count").desc).limit(10).show()